In [82]:
pip install apyori

Note: you may need to restart the kernel to use updated packages.


In [83]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

from apyori import apriori
from mlxtend. frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import datetime

In [ ]:
df = pd.read_excel('data_retail2.xlsx')

In [95]:
df.head()

,InvoiceNo,InvoiceDate,BRANCH_SPLR,BRANCHNAME_SPLR,warehouseProductsID,BARCODEID,StockCode,PRODUCT,PRODUCT_CATEGORY,Quantity,...,CustomerID,CUSTNAME,ADDRESS,KOTA,PROVINSI,NEGARA,CHANNELID_SPLR,CHANNELNAME_SPLR,SUBDISTID,SUBDIST_NAME
0,536365,2020-12-01 08:26:00,13,SAMARINDA,10001,8992753282401,85123A,123 BENDERA COKLAT 300G,SUSU,6,...,17850.0,DUA PUTRI SLAMET RIYADI,JL. SLAMET RIYADI,SAMARINDA,KALIMANTAN TIMUR,INDONESIA,32,Toko Kelontong,130113,CV. EKA PUTRA
1,536365,2020-12-01 08:26:00,13,SAMARINDA,20020,8999909192034,71053,2.3.4 FILTER,ROKOK,6,...,17850.0,SANURI,JL. M. SAID,SAMARINDA,KALIMANTAN TIMUR,INDONESIA,32,Toko Kelontong,130113,CV. EKA PUTRA
2,536365,2020-12-01 08:26:00,19,YOGYAKARTA,20021,8999909007147,84406B,234 KERETEK,ROKOK,8,...,17850.0,EMI MBAK,PS. TALUN NO.63,MAGELANG,JAWA TENGAH,INDONESIA,32,Toko Kelontong,190105,PT. KTRI DISTRIBUSI
3,536365,2020-12-01 08:26:00,13,SAMARINDA,20018,8999909172234,84029G,234 KERETEK 12,ROKOK,6,...,17850.0,RISKA CELL ADAM MALIK,JL. ADAM MALIK,SAMARINDA,KALIMANTAN TIMUR,INDONESIA,32,Toko Kelontong,130113,CV. EKA PUTRA
4,536365,2020-12-01 08:26:00,19,YOGYAKARTA,20019,8999909005860,84029E,234 KERETEK 16,ROKOK,6,...,17850.0,ABADI MART,JL.GROWONG - PUCUNG REJO MUNTILAN.,MAGELANG,JAWA TENGAH,INDONESIA,42,Mini Market,190105,PT. KTRI DISTRIBUSI


In [96]:
# Mengganti kolom PERIODE menjadi tipe data datetime 
df['InvoiceDate'] = pd.to_datetime (df['InvoiceDate'])

In [97]:
#Menghapus column yang tidak digunakan / duplicate columns 
#df = df.drop(columns=['SALES_HNA RUPIAH', 'SALES_HNA_UNIT', 'NET SALES', 'NET_SALES_BU'])

In [98]:
 # Clean up spaces in product description, product category and remove any rows that don't have a valid invoice 
df['PRODUCT'] = df['PRODUCT'].str.strip()
df['PRODUCT_CATEGORY'] = df['PRODUCT_CATEGORY'].str.strip()

df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

In [99]:
# menghapus variabel inoviceNO yang diawali dengan huruf C pada invoice numbernya
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~(df['InvoiceNo'].str[0] == 'c')]

In [100]:
basket = (df[df['PROVINSI'] =="JAWA TENGAH"].groupby(['InvoiceNo', 'PRODUCT_CATEGORY']) ['Quantity'].count()\
                               .unstack().reset_index().fillna(0)\
                               .set_index('InvoiceNo'))
basket.head()

PRODUCT_CATEGORY,ALAT LISTRIK,ALAT RUMAH TANGGA,BEER,BERAS,BISKUIT,BUMBU,COKELAT,DETERGEN,DIET FOOD,ELEKTRONIK,...,SEMIR SEPATU,SLAI/JAM,SNACK,STATIONERY,SUSU,SYRUP,TANDAS,TEH & KOPI,TEPUNG,TISSUE
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536373,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [101]:
# Show a subset of columns
basket.iloc[:,[0,1,2,3,4,5,6,7]].head()

PRODUCT_CATEGORY,ALAT LISTRIK,ALAT RUMAH TANGGA,BEER,BERAS,BISKUIT,BUMBU,COKELAT,DETERGEN
InvoiceNo,,,,,,,,
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536373,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [104]:
# Melakukan proses encoding -> Mengubah data kebentuk angka, agan sistem atau komputer dapat memanand Informasi dari dataset
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)
basket_sets.head(5)

PRODUCT_CATEGORY,ALAT LISTRIK,ALAT RUMAH TANGGA,BEER,BERAS,BISKUIT,BUMBU,COKELAT,DETERGEN,DIET FOOD,ELEKTRONIK,...,SEMIR SEPATU,SLAI/JAM,SNACK,STATIONERY,SUSU,SYRUP,TANDAS,TEH & KOPI,TEPUNG,TISSUE
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536373,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [105]:
# Build up the frequent items, the rules, and model
frequent_itemsets = apriori (basket_sets, min_support=0.1, use_colnames=True)
frequent_itemsets

C:\Users\user\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.300993,(ALAT RUMAH TANGGA)
1,0.320419,(BISKUIT)
2,0.155839,(BUMBU)
3,0.150982,(COKELAT)
4,0.246708,(DETERGEN)
...,...,...
147,0.106626,"(SUSU, SABUN & SAMPHOO, PARFUM)"
148,0.107166,"(SABUN & SAMPHOO, MINUMAN, KOSMETIK, BISKUIT)"
149,0.101230,"(SABUN & SAMPHOO, MINUMAN, KOSMETIK, OBATAN)"
150,0.107382,"(SABUN & SAMPHOO, MINUMAN, KOSMETIK, PARFUM)"


In [108]:
rules1 = association_rules(frequent_itemsets, metric="lift", min_threshold=1) 
rules1.head() # 1280 rows 9 columns

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(ALAT RUMAH TANGGA),(BISKUIT),0.300993,0.320419,0.157889,0.524561,1.637110,0.061445,1.429376,0.556743
1,(BISKUIT),(ALAT RUMAH TANGGA),0.320419,0.300993,0.157889,0.492759,1.637110,0.061445,1.378056,0.572658
2,(ALAT RUMAH TANGGA),(DETERGEN),0.300993,0.246708,0.124218,0.412693,1.672796,0.049960,1.282619,0.575385
3,(DETERGEN),(ALAT RUMAH TANGGA),0.246708,0.300993,0.124218,0.503500,1.672796,0.049960,1.407868,0.533921
4,(ALAT RUMAH TANGGA),(KOSMETIK),0.300993,0.380747,0.161343,0.536034,1.407850,0.046740,1.334696,0.414441


In [111]:
result1 = rules1[(rules1['lift'] >= 1) &
                (rules1['confidence'] >= 0.8)]

apr_result = result1.sort_values (by='confidence', ascending=False)
apr_result.head(20) # 64 rows 9 columns

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
523,"(SABUN & SAMPHOO, MINUMAN, PARFUM)",(KOSMETIK),0.123138,0.380747,0.107382,0.872042,2.290346,0.060497,4.839506,0.642501
509,"(SABUN & SAMPHOO, MINUMAN, OBATAN)",(KOSMETIK),0.117419,0.380747,0.101230,0.862132,2.264319,0.056524,4.491650,0.632651
536,"(SUSU, SABUN & SAMPHOO, MINUMAN)",(KOSMETIK),0.119577,0.380747,0.103065,0.861913,2.263744,0.057536,4.484526,0.634075
495,"(SABUN & SAMPHOO, MINUMAN, BISKUIT)",(KOSMETIK),0.125728,0.380747,0.107166,0.852361,2.238654,0.059295,4.194360,0.632873
510,"(SABUN & SAMPHOO, KOSMETIK, OBATAN)",(MINUMAN),0.120980,0.349558,0.101230,0.836753,2.393749,0.058941,3.984404,0.662380
537,"(SUSU, SABUN & SAMPHOO, KOSMETIK)",(MINUMAN),0.125081,0.349558,0.103065,0.823986,2.357226,0.059342,3.695406,0.658087
381,"(SABUN & SAMPHOO, OBATAN)",(KOSMETIK),0.147205,0.380747,0.120980,0.821848,2.158514,0.064932,3.475973,0.629364
447,"(SUSU, OBATAN)",(MINUMAN),0.134038,0.349558,0.109864,0.819646,2.344809,0.063010,3.606471,0.662300
423,"(SABUN & SAMPHOO, STATIONERY)",(KOSMETIK),0.133175,0.380747,0.108461,0.814425,2.139019,0.057755,3.336936,0.614306
410,"(PERMEN, SABUN & SAMPHOO)",(KOSMETIK),0.128966,0.380747,0.104900,0.813389,2.136299,0.055796,3.318420,0.610655
